참고  :https://github.com/znxlwm/tensorflow-MNIST-GAN-DCGAN/

In [6]:
import os, time, imageio, itertools, pickle
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

# 데이터 불러오기

In [7]:
# Load data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
train_set = mnist.train.images
mean_train = np.mean(train_set)
std_train =np.std(train_set)
train_set = (train_set - 0.5)/0.5

print("data shape : ", train_set.shape)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
data shape :  (55000, 784)


# 필요한 함수 정의

In [8]:
# relu 함수
def relu_layer(x,node,std,name="relu_layer",drop_out=0.0):
    with tf.variable_scope(name):
        w = tf.get_variable("w",[x.get_shape()[1],node],initializer=tf.truncated_normal_initializer(mean=0.,stddev=std))
        b = tf.get_variable("b",[node],initializer=tf.constant_initializer(0.))
        if drop_out!=0:
            return tf.nn.dropout(tf.nn.relu(tf.matmul(x,w)+b),drop_out)
        else:
            return tf.nn.relu(tf.matmul(x,w)+b)
        

        
# 훈련된 과정을 볼 수 있는 함수.        
def show_train_hist(hist,show=False,save=False,path='Train_hist.png'):
    x = range(len(hist['D_losses']))
    
    y1 = hist['D_losses']
    y2 = hist['G_losses']
    plt.subplots(1,1,figsize=(5,5))
    plt.plot(x,y1,label='D_loss')
    plt.plot(x,y2,label='G_loss')
    
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()
    
    if save:
        plt.savefig(path)
        
    if show:
        plt.show()
    else:
        plt.close()
        


# GAN정의

In [9]:
class GAN(object):
    def __init__(self,sess,input_size=784,output_size=784,
                 batch_size = 128,z_dim = 100,
                 gf_dim = 254,df_dim=1024):
        '''
        Args:
            sess : Tensorflow session
            batch_size : The size of batch
            z_dim : Dimension of 
            gf_dim : Dimension of generator filters in first genrator layers
            df_dim : Dimension of discriminator filters in first discriminator layers
            
        '''
        self.sess = sess
        self.input_size = input_size
        self.output_size = output_size
        self.batch_size = 128
        self.z_dim = z_dim
        self.gf_dim = gf_dim
        self.df_dim = df_dim
        self.build_model()

    # model을 생성한다.
    def build_model(self):

        self.x = tf.placeholder('float32',[self.batch_size,self.input_size],'real_x')
        self.z = tf.placeholder('float32',[None,self.z_dim],'z')
        self.drop_out = tf.placeholder('float32',name='drop_out')
        
        self.G = self.generator(self.z)
        self.D_real = self.discriminator(self.x,self.drop_out,reuse=False)
        self.D_fake = self.discriminator(self.G,self.drop_out,reuse=True)
        

        eps = 1e-2
        self.d_loss = tf.reduce_mean(-tf.log(self.D_real+eps)-tf.log(1-self.D_fake+eps))
        self.g_loss = tf.reduce_mean(-tf.log(self.D_fake+eps))
        
        t_vars = tf.trainable_variables()
        
        self.d_vars = [var for var in t_vars if 'discriminator' in var.name]
        self.g_vars = [var for var in t_vars if 'generator' in var.name]
        
    # training
    def train(self,data,epoch=50,learning_rate=0.0002,beta1=0.9,drop_out=0.5):
        
        self.data = data
        self.learning_rate = learning_rate
        self.bata1 = beta1
        self.epoch = epoch
        
        self.d_optim = tf.train.AdamOptimizer(learning_rate,beta1=beta1).minimize(self.d_loss,var_list = self.d_vars)
        self.g_optim = tf.train.AdamOptimizer(learning_rate,beta1=beta1).minimize(self.g_loss,var_list = self.g_vars)
        
        init = tf.global_variables_initializer()
        self.sess.run(init)
        
        sample_z = np.random.normal(0,1,(25,100))
        iter_per_epoch = len(self.data) // self.batch_size
        counter =1 
        self.epoch = epoch
        self.train_hist={}
        self.train_hist['D_losses']=[]
        self.train_hist['G_losses']=[]
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime']=[]
        self.train_hist['epoch'] = []
        start_time = time.time()
        
        # results save folder
        if not os.path.isdir("GAN_results"):
            os.mkdir('GAN_results')
        if not os.path.isdir("GAN_results/images"):
            os.mkdir("GAN_results/images")

        for e in range(self.epoch):
            epoch_start_time =time.time()
            for idx in range(iter_per_epoch):
                start_ptime = time.time()
                batch_images = self.data[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_z = np.random.normal(0,1,(self.batch_size,100))
                
                # update D network
                _, d_loss = self.sess.run([self.d_optim,self.d_loss],feed_dict={self.x:batch_images,self.z:batch_z,self.drop_out:drop_out})
                
                # update G netework
                _, g_loss = self.sess.run([self.g_optim,self.g_loss],feed_dict={self.z:batch_z,self.drop_out:drop_out})
                counter+=1
                

                        
            print("Epoch:[%2d] time: %4.4f, d_loss : %.8f, g_loss: %.8f"%
                    (e,time.time()-epoch_start_time,d_loss,g_loss))
            
            self.train_hist['D_losses'].append(d_loss)
            self.train_hist['G_losses'].append(g_loss)
            self.train_hist['per_epoch_ptimes'].append(time.time()-epoch_start_time)
            self.train_hist['total_ptime'].append(time.time()-start_time)
            self.train_hist['epoch']
            
            save_image = self.sess.run(self.G,feed_dict={self.z:sample_z})
            
            save_image = np.reshape(save_image,[25,int(np.sqrt(self.input_size)),int(np.sqrt(self.input_size))])
            
            # 표시할 그림 수 
            size_figure_grid=5
            # fig는 나타내는 큰 틀
            # ax는 fig에 포함된 그림
            fig,ax = plt.subplots(size_figure_grid,size_figure_grid,figsize=(5,5))
            # ax에서 눈금을 제거
            for i,j in itertools.product(range(size_figure_grid),range(size_figure_grid)):
                ax[i,j].get_yaxis().set_visible(False)
                ax[i,j].get_yaxis().set_visible(False)
            
            # 그림을 나타내는 순서
            for k in range(5*5):
                i = k // 5
                j = k % 5
                ax[i,j].cla()
                ax[i,j].imshow(np.reshape(save_image[k],(28,28)),cmap='gray')

            path = './GAN_results/images/'
            label = 'Epoch {0}'.format(e)
            fig.text(0.5,0.04,label,ha='center')
            plt.savefig(path+str(e)+'.jpg')
            plt.close()
            
    def result(self):
        # loss function의 변화를 graph로 저장
        with open("GAN_results/train_hist.pkl",'wb') as f:
            pickle.dump(self.train_hist,f)
        show_train_hist(self.train_hist, save=True, path='GAN_results/GAN_train_hist.png')

        self.images=[]
        # image들을 gif파일로 저장
        for e in range(self.epoch):
            image_name = './GAN_results/images/'+str(e)+'.jpg'
            self.images.append(imageio.imread(image_name))

        imageio.mimsave('GAN_results/generation_animation.gif',self.images,fps=10)

        
    def generator(self,z):
        with tf.variable_scope('generator'):
            # hidden 1
            hidden1 = relu_layer(x = z,node = 254, std = 0.02,name="G_h1")

            # hidden 2
            hidden2 = relu_layer(x = hidden1,node = 512,std = 0.02,name="G_h2")

            # hidden 3
            hidden3 = relu_layer(x = hidden2,node = 1024,std = 0.02,name="G_h3")

            # hidden 4
            with tf.variable_scope("G_out"):
                w = tf.get_variable('w',[hidden3.get_shape()[1],784],initializer=tf.truncated_normal_initializer(mean=0,stddev=0.02))
                b = tf.get_variable("b",[1],initializer = tf.constant_initializer(0.))
                out = tf.nn.tanh(tf.matmul(hidden3,w)+b)
            return out        
    
    def discriminator(self,x,drop_out,reuse=False):
        with tf.variable_scope('discriminator') as scope:
            if reuse:
                scope.reuse_variables()
                
            # hidden 1
            hidden1 = relu_layer(x = x,node = self.df_dim,std = 0.02,drop_out=drop_out,name='D_h1')

            # hidden 2
            hidden2 = relu_layer(x = hidden1,node = int(self.df_dim/2),std = 0.02,drop_out=drop_out,name="D_h2")

            # hidden 3
            hidden3 = relu_layer(x = hidden2,node = int(self.df_dim/4),std = 0.02,drop_out = drop_out,name='D_h3')

            # hdden 4
            with tf.variable_scope("D_out"):
                w = tf.get_variable("w",[hidden3.get_shape()[1],1],initializer=tf.truncated_normal_initializer(mean=0,stddev=0.02))
                b = tf.get_variable("b",[1],initializer = tf.constant_initializer(0.))
                out = tf.sigmoid(tf.matmul(hidden3,w)+b)

            return out
                             
                        
    
        
    

# 하이퍼파라미터

In [1]:
batch_size= 256
learning_rate = 0.0002
train_epoch = 300

# 실행

In [11]:
tf.reset_default_graph()

gan = GAN(sess=tf.Session())

gan.train(train_set)

gan.result()


Epoch:[ 0] time: 4.0415, d_loss : 0.25609130, g_loss: 3.18347621


/usr/local/lib/python3.5/dist-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Epoch:[ 1] time: 3.7699, d_loss : 0.05877637, g_loss: 3.78600621
Epoch:[ 2] time: 3.9666, d_loss : -0.01364723, g_loss: 4.52642441
Epoch:[ 3] time: 3.8861, d_loss : 0.10865010, g_loss: 3.32130837
Epoch:[ 4] time: 3.8401, d_loss : 0.25496328, g_loss: 3.05040359
Epoch:[ 5] time: 3.8045, d_loss : 0.11699235, g_loss: 2.98142338
Epoch:[ 6] time: 3.9359, d_loss : 0.38123462, g_loss: 2.56508470
Epoch:[ 7] time: 3.8210, d_loss : 0.46453840, g_loss: 2.43334985
Epoch:[ 8] time: 3.8285, d_loss : 0.49145687, g_loss: 2.99119020
Epoch:[ 9] time: 3.9799, d_loss : 0.39742225, g_loss: 2.33363533
Epoch:[10] time: 3.9547, d_loss : 0.51886845, g_loss: 2.33598852
Epoch:[11] time: 3.8762, d_loss : 0.26577196, g_loss: 2.65595555
Epoch:[12] time: 3.8673, d_loss : 0.40984327, g_loss: 2.69819736
Epoch:[13] time: 3.9042, d_loss : 0.51845384, g_loss: 2.13338947
Epoch:[14] time: 3.7864, d_loss : 0.47256237, g_loss: 2.45844460
Epoch:[15] time: 3.8526, d_loss : 0.55647147, g_loss: 2.15216899
Epoch:[16] time: 3.9184,